In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [2]:
#load the trained models
model = load_model('model.h5')

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
with open('onehotencoder_geo.pkl', 'rb') as file:
    onehotencoder_geo = pickle.load(file)
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)


In [10]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}


In [11]:
geo_encoded = onehotencoder_geo.transform([[input_data['Geography']]]).toarray()
d_df = pd.DataFrame(geo_encoded, columns=onehotencoder_geo.get_feature_names_out())

/Users/rohandasari/Prep/DLproject/customer_churn/DLenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [12]:
d_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
input_data = pd.DataFrame([input_data])

In [14]:
input_data = pd.concat([input_data.reset_index(drop=True), d_df.reset_index(drop=True)], axis=1)
input_data = input_data.drop(['Geography'], axis=1)

In [15]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data.head() 

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
input_scaled = scaler.transform(input_data)

In [17]:
pred = model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [22]:
pred[0][0]

np.float32(0.044294655)

In [23]:
if(pred[0][0] == 1):
    print("The customer is likely to leave the bank.")
else:       
    print("The customer is likely to stay with the bank.")

The customer is likely to stay with the bank.
